In [33]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import pylab

In [34]:
# Load up codebook
pc = pd.DataFrame.from_csv('cap-data/recs2009_public_codebook_simple.csv', index_col= None, encoding = 'utf-8')

In [35]:
# Separate response codes and labels
pc['Response Codes']=[x.split('\r') if type(x) == str else x for x in pc['Response Codes']]
pc['Response Labels']=[x.split('\r') if type(x) == str else x for x in pc['Response Labels']]
pc = pc.ix[:930] # last 3 lines unnecessary

In [36]:
# Imputations are the EIA's guesses at filling in blank forms. Since blanks are few, imputation will be ignored.
no_imp = pc[pc['Variable Description'].str.contains('Imputation flag for') != True]
pc = no_imp

In [56]:
pc[pc['Variable Description'].str.contains('usage')]


,Variable Name,Variable Description,Response Codes,Response Labels,Electricity,Natural Gas,Propane,Fuel Oil,Kerosene
124,AMTMICRO,Microwave usage,"[1, 2, 3, 4, -2]","[Used to cook or reheat most meals and snacks,...",NaN,NaN,NaN,NaN,NaN
252,TVONWD1,Most-used TV usage on weekdays,"[1, 2, 3, 4, 5, -2]","[Less than 1 hour, 1 to 3 hours, 3 to 6 hours,...",OTH_A,NaN,NaN,NaN,NaN
253,TVONWDWATCH1,Most-used TV weekday usage spent playing video...,"[0, 1, 2, 3, 4, -2]","[None, All, Most, Half, Little, Not Applicable]",NaN,NaN,NaN,NaN,NaN
254,TVONWE1,Most-used TV usage on weekends,"[1, 2, 3, 4, 5, -2]","[Less than 1 hour, 1 to 3 hours, 3 to 6 hours,...",OTH_A,NaN,NaN,NaN,NaN
255,TVONWEWATCH1,Most-used TV weekend usage spent playing video...,"[0, 1, 2, 3, 4, -2]","[None, All, Most, Half, Little, Not Applicable]",NaN,NaN,NaN,NaN,NaN
268,TVONWD2,Second most-used TV usage on weekdays,"[1, 2, 3, 4, 5, -2]","[Less than 1 hour, 1 to 3 hours, 3 to 6 hours,...",OTH_A,NaN,NaN,NaN,NaN
269,TVONWDWATCH2,Second most-used TV weekday usage spent playin...,"[0, 1, 2, 3, 4, -2]","[None, All, Most, Half, Little, Not Applicable]",NaN,NaN,NaN,NaN,NaN
270,TVONWE2,Second most-used TV usage on weekends,"[1, 2, 3, 4, 5, -2]","[Less than 1 hour, 1 to 3 hours, 3 to 6 hours,...",OTH_A,NaN,NaN,NaN,NaN
271,TVONWEWATCH2,Second most-used TV weekend usage spent playin...,"[0, 1, 2, 3, 4, -2]","[None, All, Most, Half, Little, Not Applicable]",NaN,NaN,NaN,NaN,NaN
284,TVONWD3,Third most-used TV usage on weekdays,"[1, 2, 3, 4, 5, -2]","[Less than 1 hour, 1 to 3 hours, 3 to 6 hours,...",OTH_A,NaN,NaN,NaN,NaN


In [71]:
pc['Response Labels'][0][0].isin('kilowatt')

AttributeError: 'str' object has no attribute 'isin'

In [38]:
# # Separate numeric responses
# num_resp = no_imp[no_imp['Response Codes'].isnull() == True]
# num_resp

In [39]:
# # Separate categorical responses
# cat_resp = no_imp[no_imp['Response Codes'].isnull() == False]
# cat_resp

In [41]:
hold = pc.reset_index()
del hold['index']
hold

,Variable Name,Variable Description,Response Codes,Response Labels,Electricity,Natural Gas,Propane,Fuel Oil,Kerosene
0,DOEID,Unique identifier for each respondent,[00001 - 12083],[Unique identifier for each respondent],NaN,NaN,NaN,NaN,NaN
1,REGIONC,Census Region,"[1, 2, 3, 4]","[Northeast Census Region, Midwest Census Regio...",NaN,NaN,NaN,NaN,NaN
2,DIVISION,Census Division,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[New England Census Division (CT, MA, ME, NH, ...",NaN,NaN,NaN,NaN,NaN
3,REPORTABLE_DOMAIN,Reportable states and groups of states,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[, Connecticut, Maine, New Hampshire, Rhode Is...",NaN,NaN,NaN,NaN,NaN
4,TYPEHUQ,Type of housing unit,"[1, 2, 3, 4, 5]","[Mobile Home, Single-Family Detached, Single-F...",OTH_B,SPH_A,NaN,NaN,NaN
5,NWEIGHT,Final sample weight,NaN,[Final sample weight],NaN,NaN,NaN,NaN,NaN
6,HDD65,"Heating degree days in 2009, base temperature 65F",NaN,"[Heating degree days in 2009, base temperature...","SPH_B, WTH_A",SPH_B,"SPH_B, OTH_B","SPH_B, OTH_B","SPH_B, OTH_B"
7,CDD65,"Cooling degree days in 2009, base temperature 65F",NaN,"[Cooling degree days in 2009, base temperature...",COL_B,NaN,NaN,NaN,NaN
8,HDD30YR,"Heating degree days, 30-year average 1981-2010...",NaN,"[Heating degree days, 30-year average 1981-201...",NaN,NaN,NaN,NaN,NaN
9,CDD30YR,"Cooling degree days, 30-year average 1981-2010...",NaN,"[Cooling degree days, 30-year average 1981-201...",NaN,NaN,NaN,NaN,NaN


In [42]:
#Now there are variables that are the same thing with different units. We'll attempt to sort out those duplicates here.

dups = pc.reset_index()
del dups['index']
dups['Split Description']=[x.split(',') for x in dups['Variable Description']]
#dups['Variable Description']=[x.split(',') if x.find(',') != 0 else x for x in dups['Variable Description']]

In [44]:
# dummy = pd.DataFrame(columns = ['full_description','broken_up','first_part'])
dummy = pd.DataFrame(columns = ['first_part'])
# dummy['full_description'] = no_imp['Variable Description']
# dummy['broken_up'] = dups['Variable Description']

i = 0
for cell in dups['Split Description']:
        dummy.loc[len(dummy)] = cell[0]

        

        
dummy['full_description'] = dups['Variable Description']
dummy['broken_up'] = dups['Split Description']
dummy = dummy.sort_values(by = 'first_part')
# dummy = dummy.reset_index()
dummy

,first_part,full_description,broken_up
468,2009 gross household income,2009 gross household income,[2009 gross household income]
11,AIA Climate Zone,"AIA Climate Zone, based on average temperature...","[AIA Climate Zone, based on average temperatu..."
29,Addition to a mobile home,Addition to a mobile home,[Addition to a mobile home]
450,Age category of eighth household member,Age category of eighth household member,[Age category of eighth household member]
453,Age category of eleventh household member,Age category of eleventh household member,[Age category of eleventh household member]
447,Age category of fifth household member,Age category of fifth household member,[Age category of fifth household member]
456,Age category of fourteenth household member,Age category of fourteenth household member,[Age category of fourteenth household member]
446,Age category of fourth household member,Age category of fourth household member,[Age category of fourth household member]
451,Age category of ninth household member,Age category of ninth household member,[Age category of ninth household member]
444,Age category of second household member,Age category of second household member,[Age category of second household member]


In [45]:
# This weeds out parameters that are repeated with different units
first_repeats = pd.DataFrame(columns = ['full_description','first_part'])

i = 0
for cell in dummy.first_part:
    if i < 571:
        if cell == dummy.first_part.iloc[i+1]:
            first_repeats.loc[len(first_repeats)] = [dummy.full_description.iloc[i], dummy.first_part.iloc[i]]
    i = i + 1


In [46]:
first_repeats

,full_description,first_part
0,"Electricity usage for air-conditioning, centra...",Electricity usage for air-conditioning
1,Electricity usage for other purposes (all end-...,Electricity usage for other purposes (all end-...
2,"Electricity usage for refrigerators, in kilowa...",Electricity usage for refrigerators
3,"Electricity usage for space heating, in kilowa...",Electricity usage for space heating
4,"Electricity usage for water heating, in thousa...",Electricity usage for water heating
5,Fuel Oil usage for other purposes (all end-use...,Fuel Oil usage for other purposes (all end-use...
6,"Fuel Oil usage for space heating, in gallons, ...",Fuel Oil usage for space heating
7,"Fuel Oil usage for water heating, in thousand ...",Fuel Oil usage for water heating
8,Kerosene usage for other purposes (all end-use...,Kerosene usage for other purposes (all end-use...
9,"Kerosene usage for space heating, in gallons, ...",Kerosene usage for space heating


In [49]:
# Now drop identified duplicate columns from codebook
pc['Variable Description']  first_repeats.full_description

ValueError: Series lengths must match to compare

In [14]:
# Load up data
data = pd.DataFrame.from_csv('cap-data/recs2009_public.csv', index_col = None, encoding = 'utf-8')